#### Proyecto CAPSTONE - Felipe González García
# Sistema de Recuperación de Información (RAG) para Contact Center Corporativo

## 1. Pasos Previos

Se instalan las librerías necesarias, como gspread, pandas, openai, langchain, numpy, y scikit-learn

In [6]:
!pip install --quiet gspread google-auth pandas openai langchain langchain_openai numpy scikit-learn

Se autentica al usuario con Google para acceder a la hoja de cálculo.

In [7]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

from google.colab import userdata

Se abre la hoja de cálculo, se selecciona la hoja relevante y se cargan los datos en un DataFrame de Pandas.

In [8]:
import gspread
import pandas as pd # Importamos el módulo Pandas

# Nos autenticamos en Google (IDP)
gc = gspread.authorize(creds)

# Abrimos el Google Sheet y seleccionamos la hoja de cálculo
hoja_de_calculo = gc.open_by_key(userdata.get('SHEET_ID'))
hoja = hoja_de_calculo.worksheet('DATOS')

# Obtiene los datos como lista de diccionarios
data = hoja.get_all_records()
df = pd.DataFrame(data)

from IPython.display import display
display(df)


,DEPARTAMENTO,CONSULTA,SOLUCIÓN,ENLACES DE INTERÉS
0,ADMINISTRACIÓN (FACTURACIÓN) 551010,Enviar facturas,Prioritariamente nos preguntamos porqué no ha ...,
1,ADMINISTRACIÓN (FACTURACIÓN) 551010,Facturación incorrecta,Prioritariamente verificamos si se trata de un...,
2,ADMINISTRACIÓN (FACTURACIÓN) 551010,Aclaración de factura,Pregunto sobre qué factura sería la consulta. ...,
3,ADMINISTRACIÓN (FACTURACIÓN) 551010,Cambio numero de cuenta,"Accedo a ofertas PRL en su pestaña general, pa...",https://drive.google.com/file/d/1Ooo-ZgPmfMxKb...
4,ADMINISTRACIÓN (FACTURACIÓN) 551010,Cambio forma de pago,CRM/Ofertas prl/Contratos aceptados/General: c...,
...,...,...,...,...
125,INFORMACIÓN AUXILIAR,Fusión preving-cualtis,Empresas que solicitan documentación sobre la...,https://drive.google.com/file/d/1xERboEfR8PGgg...
126,INFORMACIÓN AUXILIAR,LLamadas de compañeros,Cuando recibamos llamadas de compañeros indica...,
127,INFORMACIÓN AUXILIAR,Ofertas de nuevos servicios. Proveedores,En el caso de empresas que deseen ofrecer sus ...,
128,INFORMACIÓN AUXILIAR,CIF Vitaly,,https://drive.google.com/file/d/1VGKJHI1wgFAu2...


Definir el "Documento": Decidir qué columnas combinadas formarán el texto que representa cada procedimiento (el "documento" a buscar).
Se crea una columna "documento_completo" que combina la información relevante de varias columnas para formar el texto que representa cada procedimiento.

In [9]:
# Asumiendo columnas 'Departamento', 'Necesidad', 'Procedimiento', 'Enlaces de Interés'
df['documento_completo'] = "DEPARTAMENTO: " + df['DEPARTAMENTO'] + " // CONSULTA: " + df['CONSULTA'] + " // SOLUCION: " + df['SOLUCIÓN'] + " // ENLACES DE INTERÉS: " + df['ENLACES DE INTERÉS']
documents = df['documento_completo'].tolist()

print(documents)


['DEPARTAMENTO: ADMINISTRACIÓN (FACTURACIÓN) 551010 // CONSULTA: Enviar facturas // SOLUCION: Prioritariamente nos preguntamos porqué no ha recibido la/s factura/s. Verificamos el mail de envío y aludimos a la extranet como soporte. \nBuscamos la fra. en la pestaña e-fra. de CRM, la descargamos en PDF y se la hacemos llegar al cliente por mail de Genesys. Verificamos la recepción del envío. \nEn el caso de facturas no volcadas en CRM, cerramos tipificando PBO Envio y consulta y facturas\n\nIMPORTANTE: Tipificamos tanto la llamada, como el mail que ha motivado el envío de la fra. // ENLACES DE INTERÉS: ', 'DEPARTAMENTO: ADMINISTRACIÓN (FACTURACIÓN) 551010 // CONSULTA: Facturación incorrecta // SOLUCION: Prioritariamente verificamos si se trata de una facturaciñon incorrecta, con lo que nos dirigimos a la opción de Ofertas PRL u Otras ofertas de CRM y verificamos los términos de contrato: servicios contratados, periodo de vigencia, términos de pago y de facturación.\n\n Además, consultam

Se generan embeddings para cada procedimiento utilizando el modelo text-embedding-3-small de OpenAI

In [10]:
import os
from langchain_openai import OpenAIEmbeddings

# Set the OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_APIKEY')

# Seleccionamos el modelo de embeddings
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Generar embeddings para todos los documentos
document_embeddings = embeddings_model.embed_documents(documents)

Implementación del RAG Básico (Núcleo de la PoC) : Se define una función **find_similar_documents** que busca los procedimientos más similares a una consulta dada utilizando la similitud del coseno.

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Encuentra los docuementos más similares a la pregunta de un usuario
def find_similar_documents(query, top_n=3):
  # Convierte la pregunta en un embedding o representación numérica
  query_embedding = embeddings_model.embed_query(query)
  # Calcular similitud entre el embedding de la consulta y todos los embeddings de documentos
  similarities = cosine_similarity([query_embedding], document_embeddings)[0]
  # Obtener los índices de los más similares (coseno)
  top_indices = np.argsort(similarities)[-top_n:][::-1]
  # Devolver los textos de los documentos más relevantes
  relevant_docs = [documents[i] for i in top_indices]
  return relevant_docs

Se define una plantilla de prompt para el modelo de lenguaje gpt-4 que incluye el contexto relevante y la pregunta del usuario.

In [12]:
from langchain_openai import ChatOpenAI

# Elegimos modelo y temperatura. Valor nulo de temperatura porque queremos una respuesta más precisa
llm = ChatOpenAI(model="gpt-4", temperature=0)

from langchain_core.prompts import ChatPromptTemplate

# Plantilla de Prompt
prompt_template = """
Eres un asistente que van a usar únicamente los trabajadores (agentes) de un contact center corporativo.
Responde la pregunta del agente basándote SOLAMENTE en los siguientes procedimientos extraídos de nuestra base de datos.
Es IMPORTANTE que solamente si hay algún enlace de interés relacionado, se adjunte el mismo al final de la respuesta.
Estos enlaces de interés vendrán en el formato: 'Enlace de Interés: [URL]'. Si no hay enlaces esta bloque no aparecerá

Contexto (Procedimientos Relevantes extraídos del manual del operador):
{context}

Pregunta del Agente (un agente es el trabajador del Contact Center):
{question}

Respuesta:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)


Creación de la Cadena RAG: Se utiliza LangChain para crear una cadena que conecta los diferentes componentes del sistema RAG, incluyendo la búsqueda de información, la generación de embeddings y la generación de respuestas.

In [13]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI

# Función auxiliar para formatear los documentos recuperados
def format_docs(docs):
  """Combina la lista de documentos recuperados en un solo string."""
  return "\n\n".join(doc for doc in docs)

# Cadena RAG Simple (Usando LangChain)


# rag_chain actúa como una línea de ensamblaje:

# toma la pregunta de un usuario, encuentra información relevante en la base de conocimiento,
# prepara un prompt para el modelo de lenguaje, obtiene una respuesta y
# la entrega de vuelta en un formato amigable para el usuario.
# Este es el proceso central de un sistema RAG, que combina la recuperación
# de información con la generación de modelos de lenguaje.


# La entrada a esta cadena será la pregunta del usuario (un string)
rag_chain = (
    {
        # 1. Recuperar contexto: Llama a find_similar_documents con la pregunta de entrada,
        #    luego formatea los documentos resultantes usando format_docs.
        "context": RunnableLambda(find_similar_documents) | RunnableLambda(format_docs),

        # 2. Pasar la pregunta original: RunnablePassthrough() toma la entrada original
        #    (la pregunta del usuario) y la pasa bajo la clave "question".
        "question": RunnablePassthrough()
    }
    # 3. Llenar el prompt: El diccionario {"context": ..., "question": ...} se usa para llenar la plantilla.
    | prompt
    # 4. Llamar al LLM: El prompt formateado se envía al modelo de lenguaje.
    | llm
    # 5. Extraer la respuesta: Se obtiene el contenido de la respuesta del LLM como string.
    | StrOutputParser()
)


from IPython.display import display, HTML

# CSS para ajustar el texto
css = """
<style>
.output {
    word-wrap: break-word;
    white-space: pre-wrap;
}
</style>
"""




Pruebas puntuales

In [14]:

# Ejemplo de cómo invocar la cadena (asumiendo que `mi_consulta` es un string)
mi_consulta = "Quiere una analitica de orina"
print(mi_consulta)
print("......\n")
respuesta = rag_chain.invoke(mi_consulta)

# Mostrar la cadena con el CSS aplicado
display(HTML(css + '<div class="output">' + respuesta + '</div>'))


print("\n\n-----------------------------\n")


mi_consulta = "Ahora dice que necesita que le aclaren una factura"
print(mi_consulta)
print("......\n")
respuesta = rag_chain.invoke(mi_consulta)

display(HTML(css + '<div class="output">' + respuesta + '</div>'))


print("\n\n-----------------------------\n")


mi_consulta = "¿Con quiéen puedo hablar para que me manden la factura del mes pasado?"
print(mi_consulta)
print("......\n")
respuesta = rag_chain.invoke(mi_consulta)

display(HTML(css + '<div class="output">' + respuesta + '</div>'))


Quiere una analitica de orina
......





-----------------------------

Ahora dice que necesita que le aclaren una factura
......





-----------------------------

¿Con quiéen puedo hablar para que me manden la factura del mes pasado?
......



## 1. Chatbot
A continuación se implementa un chatbot interactivo basado en los apartados anteriores que permite a los usuarios realizar consultas y obtener respuestas del sistema RAG.

Estilos para la salida del chatbot:

In [15]:
from IPython.display import display, HTML

# CSS para ajustar el texto
css = """
<style>
.user-message {
    background-color: #f2f2f2; /* Gris claro para usuario */
    color: #333333;
    padding: 10px;
    border-radius: 5px;
    margin-bottom: 5px;
    text-align: left; /* Alinea el texto a la izquierda */
}

.bot-message {
    background-color: #e0f7fa; /* Celeste claro para chatbot */
    color: #333333;
    padding: 10px;
    border-radius: 5px;
    margin-bottom: 5px;
    text-align: left; /* Alinea el texto a la izquierda */
}

.message-text {
    margin: 0;
    font-size: 14px; /* Ajusta el tamaño de fuente */
    line-height: 1.4; /* Ajusta el interlineado */
}
</style>
"""

Funciones de ayuda para determinar cuándo salimos del chatbot, para mostrar la salida en un estilo HTML amigable, y la función que implementa el chatbot

In [16]:

def check_exit_input(user_input):
    exit_phrases = ["adios", "hasta luego", "chao", "nos vemos", "salir"]
    return any(phrase in user_input.lower() for phrase in exit_phrases)

def display_chat_message(message, sender):
    if sender == "user":
        display(HTML(css + f'<div class="user-message"><p class="message-text"><b>Usuario:</b> {message}</p></div>'))
    else:
        display(HTML(css + f'<div class="bot-message"><p class="message-text"><b>Chatbot:</b> {message}</p></div>'))

def chatbot():
       while True:

           # Obtener la entrada del usuario
           user_input = input("Introduce tu petición: ")

           # Verificar si el usuario quiere salir
           if check_exit_input(user_input):
              break

           # Obtener la respuesta del chatbot usando la cadena RAG
           response = rag_chain.invoke(user_input)

           # Mostrar la entrada del usuario y la respuesta del chatbot
           display_chat_message(user_input, "user")
           display_chat_message(response, "bot")

Ejecución del chatbot

In [17]:
chatbot()

Introduce tu petición: hola


Introduce tu petición: El cliente quiere hacer un cambio en la forma de pago


Introduce tu petición: ahora necesito informarme sobre el registro horario de adeplus


Introduce tu petición: el cliente no puede acceder a Acercate Empresas 


Introduce tu petición: chao
